## Import libraries

In [1]:
import pandas as pd

## Specify indicator, frequency, reference area, series, and reporting type

We hope that these fields will be specified by the metadata once we have managed to build this tool in to open-sdg.

In [2]:
indicator_file='indicator_1-2-1.csv'
FREQ= 'A'
REF_AREA= '826'
SERIES= 'SI_POV_NAHC'
REPORTING_TYPE= 'N'

## Import files
Here we import the indicator CSV file and the mapping we created using [ILOSTAT's SMART tool](https://www.ilo.org/ilostat/tools/smart/index.html), which is also a CSV file:

(For this example, indicator 1.2.1 is used and below you can see that it is disaggregated by Sex and Age)

In [3]:
original_csv=pd.read_csv(indicator_file)
indicator=pd.read_csv(indicator_file)
indicator=indicator.rename(columns = {"Observation status": "Obs status", "Unit multiplier":"Unit mult", "Units": "Unit measure"})
indicator.head()

,Year,Unit measure,Obs status,Unit mult,Sex,Age,Value
0,2005,Percentage (%),Normal value,Units,NaN,NaN,24.8
1,2006,Percentage (%),Normal value,Units,NaN,NaN,23.7
2,2007,Percentage (%),Normal value,Units,NaN,NaN,22.6
3,2008,Percentage (%),Normal value,Units,NaN,NaN,23.2
4,2009,Percentage (%),Normal value,Units,NaN,NaN,22.0


When the mapping was made any spaces in column names were changed to "." so this is replicated here:

In [4]:
new_columns=[]
for col in indicator.columns:
    new=col.replace(" ", ".")
    indicator=indicator.rename(columns = {col:new})
indicator.head()

,Year,Unit.measure,Obs.status,Unit.mult,Sex,Age,Value
0,2005,Percentage (%),Normal value,Units,NaN,NaN,24.8
1,2006,Percentage (%),Normal value,Units,NaN,NaN,23.7
2,2007,Percentage (%),Normal value,Units,NaN,NaN,22.6
3,2008,Percentage (%),Normal value,Units,NaN,NaN,23.2
4,2009,Percentage (%),Normal value,Units,NaN,NaN,22.0


In [5]:
indicator.tail()

,Year,Unit.measure,Obs.status,Unit.mult,Sex,Age,Value
268,2017,Percentage (%),Normal value,Units,Male,16 to 24,21.8
269,2017,Percentage (%),Normal value,Units,Male,25 to 49,15.2
270,2017,Percentage (%),Normal value,Units,Male,50 to 64,16.3
271,2017,Percentage (%),Normal value,Units,Male,65 to 74,14.3
272,2017,Percentage (%),Normal value,Units,Male,75 and over,24.1


In [6]:
mapping=pd.read_csv("121_newmapping.csv")
mapping

,DSD_Dim,DSD_Dim_Code,EXT_Dim,EXT_Dim_Code,DSD_Attri,DSD_Attri_Code,DSD_Dim_Type
0,_REPVAR_,SDG,Value,NONE,NaN,NaN,STUDYVAR
1,FREQ,A,NaN,NaN,NaN,4.0,DIM
2,REPORTING_TYPE,N,NaN,NaN,NaN,4.0,DIM
3,SERIES,SI_POV_NAHC,NaN,NaN,NaN,4.0,DIM
4,REF_AREA,826,NaN,NaN,NaN,4.0,DIM
5,SEX,F,Sex,Female,NaN,3.0,DIM
6,SEX,M,Sex,Male,NaN,3.0,DIM
7,SEX,_T,Sex,_,NaN,3.0,DIM
8,AGE,_T,Age,_,NaN,3.0,DIM
9,AGE,Y0T16,Age,15 and under,NaN,3.0,DIM


## Create some objects
Below we create a list of all the fields that could be present in an indicator file, a dictionary that contains the variables that we specified earlier and an empty dataframe which is the same length (same number of rows) as the indicator CSV file:

In [7]:
fields=['FREQ', 'REPORTING_TYPE', 'SERIES', 'REF_AREA', 'SEX', 'AGE', 'URBANISATION',
                 'INCOME_WEALTH_QUANTILE', 'EDUCATION_LEV', 'OCCUPATION', 'CUST_BREAKDOWN',
                 'COMPOSITE_BREAKDOWN', 'DISABILITY_STATUS', 'TIME_PERIOD', 'OBS_VALUE',
                 'OBS_STATUS', 'UNIT_MULT', 'UNIT_MEASURE']

dis_values={"FREQ":FREQ, "REF_AREA":REF_AREA, "SERIES":SERIES, "REPORTING_TYPE": REPORTING_TYPE}

df=pd.DataFrame(index=indicator.index)

## Populate dis_values dictionary
For each of the items in the fields list, check whether the field (capitalised and with the underscore removed) is a column name in the indicator file and whether it has already been specified in the dis_values dictionary (as it is).

If not then add the field to the dis_values dictionary as keys, with '_T' as the values:

In [8]:
for field in ['_REPVAR_' if x=="OBS_VALUE" else x for x in fields]:
    dis=field.split('_')
    dis_str=""
    for i in dis:
        dis_str += str(i).capitalize()+ " "
    dis_str=dis_str[:-1].capitalize().replace(" ", ".")
    if dis_str not in indicator.columns and field not in dis_values.keys():
        dis_values[field]='_T'

## Populate df dataframe
For each key in the dis_values dictionary create a column in df and assign the corresponding value to every row in that column:

In [9]:
for dis, value in dis_values.items():
    df[dis]=value

For each column in the indicator file create a column in df which has the corresponding column name from the mapping:

In [10]:
for col in indicator.columns:
    newcol=mapping['DSD_Dim'].loc[mapping['EXT_Dim']==col].iloc[0]
    df[newcol]=indicator[col]       
    indicator.rename(columns={col:newcol}, inplace=True)


df.head()
indicator.head()

,TIME_PERIOD,UNIT_MEASURE,OBS_STATUS,UNIT_MULT,SEX,AGE,_REPVAR_
0,2005,Percentage (%),Normal value,Units,NaN,NaN,24.8
1,2006,Percentage (%),Normal value,Units,NaN,NaN,23.7
2,2007,Percentage (%),Normal value,Units,NaN,NaN,22.6
3,2008,Percentage (%),Normal value,Units,NaN,NaN,23.2
4,2009,Percentage (%),Normal value,Units,NaN,NaN,22.0


Fill empty cells with '_' (as that what they are in the mapping):

In [11]:
df=df.fillna('_')
df.head()

,FREQ,REF_AREA,SERIES,REPORTING_TYPE,URBANISATION,INCOME_WEALTH_QUANTILE,EDUCATION_LEV,OCCUPATION,CUST_BREAKDOWN,COMPOSITE_BREAKDOWN,DISABILITY_STATUS,TIME_PERIOD,_REPVAR_,UNIT_MEASURE,OBS_STATUS,UNIT_MULT,SEX,AGE
0,A,826,SI_POV_NAHC,N,_T,_T,_T,_T,_T,_T,_T,2005,24.8,Percentage (%),Normal value,Units,_,_
1,A,826,SI_POV_NAHC,N,_T,_T,_T,_T,_T,_T,_T,2006,23.7,Percentage (%),Normal value,Units,_,_
2,A,826,SI_POV_NAHC,N,_T,_T,_T,_T,_T,_T,_T,2007,22.6,Percentage (%),Normal value,Units,_,_
3,A,826,SI_POV_NAHC,N,_T,_T,_T,_T,_T,_T,_T,2008,23.2,Percentage (%),Normal value,Units,_,_
4,A,826,SI_POV_NAHC,N,_T,_T,_T,_T,_T,_T,_T,2009,22.0,Percentage (%),Normal value,Units,_,_


For each row in df, covert current value to corresponding DSD value using mapping:

In [12]:
for i in df.index:
    for col in indicator.drop(columns=["TIME_PERIOD", "_REPVAR_"]).columns:
        df.at[i, col]=mapping['DSD_Dim_Code'].loc[mapping['EXT_Dim_Code']==df.at[i, col]].loc[mapping['DSD_Dim']==col].item()

#rename _REPVAR_ column
df.rename(columns={'_REPVAR_':'OBS_VALUE'}, inplace=True)

# sort df by disaggregation columns and reset index    
df.sort_values(list(indicator.drop(columns=["TIME_PERIOD", "_REPVAR_"]).columns), inplace=True)
df=df[fields].reset_index(drop=True)
df.head()

,FREQ,REPORTING_TYPE,SERIES,REF_AREA,SEX,AGE,URBANISATION,INCOME_WEALTH_QUANTILE,EDUCATION_LEV,OCCUPATION,CUST_BREAKDOWN,COMPOSITE_BREAKDOWN,DISABILITY_STATUS,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,UNIT_MEASURE
0,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2005,31.4,A,0,PERCENT
1,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2006,30.5,A,0,PERCENT
2,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2007,28.6,A,0,PERCENT
3,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2008,29.7,A,0,PERCENT
4,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2009,28.9,A,0,PERCENT


## Write df to XML
We start by writing what is the header of the xml file created when we converted the CSV using ILOSTAT's SMART tool:

(Hopefully this will be generated automatically in the future so this step won't be needed)

In [13]:
# write top of xml to string called indicator_xml
indicator_xml='<?xml version="1.0" encoding="UTF-8"?>\n'
indicator_xml+='<message:StructureSpecificData xmlns:ss="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/structurespecific" xmlns:footer="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message/footer"\n'
indicator_xml+='xmlns:ns1="urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=UNSD:SDG(0.4):ObsLevelDim:TIME_PERIOD" xmlns:message="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message"\n'
indicator_xml+='xmlns:common="http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common"\n'
indicator_xml+='xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xml="http://www.w3.org/XML/1998/namespace">\n'
indicator_xml+='  <message:Header>\n'
indicator_xml+='    <message:ID>IDREF636797685310003267</message:ID>\n'
indicator_xml+='    <message:Test>false</message:Test>\n'
indicator_xml+='    <message:Prepared>2010-02-04T08:35:31</message:Prepared>\n'
indicator_xml+='    <message:Sender id="ONS"/>\n'
indicator_xml+='    <message:Structure structureID="UNSD_SDG_1_0" namespace="urn:sdmx:org.sdmx.infomodel.datastructure.DataStructure=UNSD:SDG(1.0):ObsLevelDim:TIME_PERIOD" dimensionAtObservation="TIME_PERIOD">\n'
indicator_xml+='      <common:Structure>\n'
indicator_xml+='        <Ref id="SDG" agencyID="UNSD" version="0.4"/>\n'
indicator_xml+='      </common:Structure>\n'
indicator_xml+='    </message:Structure>\n'
indicator_xml+='  </message:Header>\n'
indicator_xml+='  <message:DataSet ss:dataScope="DataStructure" xsi:type="ns1:DataSetType" ss:structureRef="UNSD_SDG_0_4">\n'


Get unique disaggregations combinations from df:

In [14]:
unique=df.iloc[:,:13].drop_duplicates()
unique

,FREQ,REPORTING_TYPE,SERIES,REF_AREA,SEX,AGE,URBANISATION,INCOME_WEALTH_QUANTILE,EDUCATION_LEV,OCCUPATION,CUST_BREAKDOWN,COMPOSITE_BREAKDOWN,DISABILITY_STATUS
0,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T
13,A,N,SI_POV_NAHC,826,F,Y16T24,_T,_T,_T,_T,_T,_T,_T
26,A,N,SI_POV_NAHC,826,F,Y25T49,_T,_T,_T,_T,_T,_T,_T
39,A,N,SI_POV_NAHC,826,F,Y50T64,_T,_T,_T,_T,_T,_T,_T
52,A,N,SI_POV_NAHC,826,F,Y65T74,_T,_T,_T,_T,_T,_T,_T
65,A,N,SI_POV_NAHC,826,F,Y_GE75,_T,_T,_T,_T,_T,_T,_T
78,A,N,SI_POV_NAHC,826,F,_T,_T,_T,_T,_T,_T,_T,_T
91,A,N,SI_POV_NAHC,826,M,Y0T16,_T,_T,_T,_T,_T,_T,_T
104,A,N,SI_POV_NAHC,826,M,Y16T24,_T,_T,_T,_T,_T,_T,_T
117,A,N,SI_POV_NAHC,826,M,Y25T49,_T,_T,_T,_T,_T,_T,_T


Create identifier column that is the same as the index column:

In [15]:
unique["identifier"]=unique.index
unique.head()

,FREQ,REPORTING_TYPE,SERIES,REF_AREA,SEX,AGE,URBANISATION,INCOME_WEALTH_QUANTILE,EDUCATION_LEV,OCCUPATION,CUST_BREAKDOWN,COMPOSITE_BREAKDOWN,DISABILITY_STATUS,identifier
0,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,0
13,A,N,SI_POV_NAHC,826,F,Y16T24,_T,_T,_T,_T,_T,_T,_T,13
26,A,N,SI_POV_NAHC,826,F,Y25T49,_T,_T,_T,_T,_T,_T,_T,26
39,A,N,SI_POV_NAHC,826,F,Y50T64,_T,_T,_T,_T,_T,_T,_T,39
52,A,N,SI_POV_NAHC,826,F,Y65T74,_T,_T,_T,_T,_T,_T,_T,52


Merge identifier onto full df so that we know which rows belong to a unique disaggregation column:

In [16]:
df=df.merge(unique)
df.head()

,FREQ,REPORTING_TYPE,SERIES,REF_AREA,SEX,AGE,URBANISATION,INCOME_WEALTH_QUANTILE,EDUCATION_LEV,OCCUPATION,CUST_BREAKDOWN,COMPOSITE_BREAKDOWN,DISABILITY_STATUS,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,UNIT_MEASURE,identifier
0,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2005,31.4,A,0,PERCENT,0
1,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2006,30.5,A,0,PERCENT,0
2,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2007,28.6,A,0,PERCENT,0
3,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2008,29.7,A,0,PERCENT,0
4,A,N,SI_POV_NAHC,826,F,Y0T16,_T,_T,_T,_T,_T,_T,_T,2009,28.9,A,0,PERCENT,0


Get observation values from df:

In [17]:
obs_values=df.iloc[:,13:]
obs_values.head()

,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,UNIT_MEASURE,identifier
0,2005,31.4,A,0,PERCENT,0
1,2006,30.5,A,0,PERCENT,0
2,2007,28.6,A,0,PERCENT,0
3,2008,29.7,A,0,PERCENT,0
4,2009,28.9,A,0,PERCENT,0


Go through unique combos and then through each observation and write each field name and value to xml_indicator:

In [18]:
for i in unique.index:
    xml='    <Series'
    for field in fields[:13]:
        xml +=" "+ unique[field].name +'="'+ unique[field][i]+'"'
    xml+=">\n"
    j=unique["identifier"][i]
    for k in obs_values.index:
        if j == obs_values['identifier'][k]:
            xml += "      <Obs"
            for field in fields[13:]:
                xml +=" "+ obs_values[field].name +'="'+ str(obs_values[field][k])+'"'
            xml+="/>\n"
    xml+="</Series>\n"
    indicator_xml+=xml

# end indicator_xml
indicator_xml+='  </message:DataSet>\n'
indicator_xml+='</message:StructureSpecificData>'

Write indicator_xml to indicator file with format "indicator_GOAL-TARGET-INDICATOR.xml":

In [19]:
with open(indicator_file.split(".")[0]+".xml", 'w') as f:
    f.write(indicator_xml)  